Convert Ananke coordinate to FIRE coordinate

In [1]:
import os
from os.path import join
import h5py

import numpy as np
import pandas as pd
import scipy.optimize as optim
from scipy.spatial.transform import Rotation as R
from sklearn.metrics import mean_squared_error as mse
from astropy import units as u
import astropy.coordinates as coord

In [2]:
gal = 'm12f'
pos_lsr = (0, 8.2, 0)
if gal == 'm12i':
    vel_lsr = (224.7092, -20.3801, 3.8954)
elif gal == 'm12f':
    vel_lsr = (226.1849, 14.3773, -4.8906)

In [3]:
# Read in merger tree, which contains the ID, FIRE coordinate, and coordinate of each star
table = pd.read_csv(f'accretion_history/stars_accretion_history_{gal}_res7100_v2.csv')
id_stars = table['id_stars'].values
pos_tree = table[['xstar', 'ystar', 'zstar']].values
pos_fire = table[['x_cart', 'y_cart', 'z_cart']].values
vel_tree = table[['vxstar', 'vystar', 'vzstar']].values
vel_fire = table[['vx_cart', 'vy_cart', 'vz_cart']].values

Convert Ananke to Lina's accretion history table

In [4]:
sim_dir = '/scratch/05328/tg846280/FIRE_Public_Simulations/ananke_subsamples/'
sim_file = f'{gal}/lsr-0-rslice-0.{gal}-res7100-md-sliced-gcat-dr2.hdf5'
with h5py.File(join(sim_dir, sim_file), 'r') as f:
#     l = np.deg2rad(f['l_true'][:])   # galactic longitude
#     b = np.deg2rad(f['b_true'][:])   # galactic lattitude
#     p = f['parallax_true'][:]
#     r = (p*u.mas).to_value(u.kpc, u.parallax())
#     pmra_cosdec = f['pmra_true'][:]
#     pmdec = f['pmdec_true'][:]
#     rv = f['radial_velocity_true'][:]

    px = f['px_true'][:]
    py = f['py_true'][:]
    pz = f['pz_true'][:]
    vx = f['vx_true'][:]
    vy = f['vy_true'][:]
    vz = f['vz_true'][:]

    parentid = f['parentid'][:]

In [5]:
# x_ananke = r * np.cos(b) * np.cos(l)
# y_ananke = r * np.cos(b) * np.sin(l)
# z_ananke = r * np.sin(b)
# pos_ananke = np.stack([x_ananke, y_ananke, z_ananke], 1)
# pos_galc = to_galactocentric(pos_ananke, pos_lsr)

pos = np.stack([px, py, pz], 1)
vel = np.stack([vx, vy, vz], 1)

rot1 = R.from_euler('Z', np.pi + np.arctan2(pos_lsr[1], pos_lsr[0]))
pos1 = rot1.apply(pos) + pos_lsr
vel1 = rot1.apply(vel) + vel_lsr

# x is off by a negative sign (?), not sure why
# TODO: FIX THIS
vel1 = vel1 * np.array([-1, 1, 1])

In [22]:
parentid

array([ 3047337,  3346948,  3346948, ..., 10385932, 10385932, 10386681],
      dtype=uint64)

Convert from Lina's merger coordinate to FIRE coordinate

In [6]:
# Define MSE loss function
def loss_func(params):    
    alpha, beta, gamma = params  # three Euler angles
    # define extrinsic rotation 
    r = R.from_euler('ZXZ', [alpha, beta, gamma], degrees=True)
    
    # apply transformation and return loss
    return mse(r.apply(pos_tree), pos_fire)

result = optim.minimize(loss_func, [45, 45, 45])
rot2 = R.from_euler('ZXZ', result.x, degrees=True)
pos2 = rot2.apply(pos1)
vel2 = rot2.apply(vel1)

In [13]:
rot2.apply(pos_tree), pos_fire

(array([[ 1.25779469, -0.52738318, -1.73828223],
        [ 2.30467258, -0.65630864,  1.51562237],
        [-1.94921207,  0.35551772, -1.50780967],
        ...,
        [-2.64448133,  2.00397144, -2.46875066],
        [-0.39061211,  0.65625325, -2.57421998],
        [-2.63675597, -1.09759486, -3.67577282]]),
 array([[ 1.26907165, -0.53589799, -1.74459004],
        [ 2.31357053, -0.66227699,  1.51455543],
        [-1.93907888,  0.3505808 , -1.50789857],
        ...,
        [-2.63531016,  1.99695407, -2.46894024],
        [-0.38296442,  0.65055506, -2.57611325],
        [-2.62787813, -1.10053763, -3.67858648]]))

Test

In [35]:
unique_parentid, counts = np.unique(parentid, return_counts=True)
id_test = unique_parentid[np.argsort(counts)[-1]]

pos_tree_test = pos_tree[id_stars==id_test]
pos_fire_test = pos_fire[id_stars==id_test]
pos1_test = pos1[parentid==id_test]
pos2_test = pos2[parentid==id_test]

vel_tree_test = vel_tree[id_stars==id_test]
vel_fire_test = vel_fire[id_stars==id_test]
vel1_test = vel1[parentid==id_test]
vel2_test = vel2[parentid==id_test]

In [36]:
pos_tree_test, pos1_test

(array([[ 3.00642776, -7.50732565, -0.08794808]]),
 array([[ 1.09409876e-01,  8.09578609e+00,  6.67659640e-02],
        [ 1.15675164e-01,  8.09501352e+00,  1.47966627e-02],
        [-4.15492418e-03,  8.21061229e+00,  1.64988236e-01],
        ...,
        [-1.41556559e-02,  8.08794515e+00,  1.06690002e-01],
        [-1.76258533e-01,  8.05167000e+00,  1.84885701e-01],
        [ 4.42617708e-02,  8.29283849e+00,  5.33285684e-02]]))

In [111]:
with h5py.File(sim_file, 'a') as f:
    for key in ('px_fire', 'py_fire', 'pz_fire', 'vx_fire', 'vy_fire', 'vz_fire'):
        if f.get(key):
            del f[key]
    f.create_dataset('px_fire', data=pos2[0])
    f.create_dataset('py_fire', data=pos2[1])
    f.create_dataset('pz_fire', data=pos2[2])
    f.create_dataset('vx_fire', data=vel2[0])
    f.create_dataset('vy_fire', data=vel2[1])
    f.create_dataset('vz_fire', data=vel2[2])